In [1]:
# Dependencies
# Dependencies
import numpy as np
#import tweepy
import time
import json
import requests
#from config import consumer_key, consumer_secret, access_token, access_token_secret
from config import api_key, gmap_key
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint

In [2]:
#Import Nutrition Facts  from file 
# load CSV
nutr_file = "Nutrition Data for Fast Food 2017.csv"
nutr_pd = pd.read_csv(nutr_file, encoding='iso-8859-1', low_memory=False)
restu_list=nutr_pd.Fast_Food_Restaurant.unique()
print(restu_list)

["McDonald's" 'Burger King' "Wendy's" 'Chick-fil-A' 'Jack in the Box'
 'Sonic' 'Dairy Queen' "Carl's Jr." "Hardee's" 'White Castle'
 'Whataburger' 'In-N-Out Burger']


In [3]:
#Write function to get city from Id
def get_city(business_id):
    #Using the business id, run a request via Yelp's Business Lookup to retrieve the city value
    yelp_data_id = requests.get(f'https://api.yelp.com/v3/businesses/{business_id}', headers=headers)
    yelp_data_id = yelp_data_id.json()
    city = yelp_data_id['location']['city']
    return city
    
# #Write function get reviews from Id
# def get_reviews(business_id)
#     yelp_data_id = requests.get(f'https://api.yelp.com/v3/businesses/{business_id}/reviews', headers=headers)
#     yelp_data_id = yelp_data_id.json()

In [ ]:
#Set header for YelpFusion API
headers = {'Authorization': 'Bearer {0}'.format(api_key)}

#set an empty list to append dictionary of each Fast Food and their rating 
restaurants_rating = []

#Loop through the restaurants to check if it exists in SF
for restaurant in restu_list:
    #Set parameters
    params = {
        "term": restaurant,
        "location": "San Francisco",
        "state": "CA",
        "country": "US",
        "limit": 15
    }

    # Build the endpoint URL (Checks all ice cream shops)
    base_url = "https://api.yelp.com/v3/businesses/search"

    # Run a request to endpoint and convert result to json
    yelp_data = requests.get(base_url, params=params, headers=headers)
    yelp_data = yelp_data.json()
        
    #Loop through all the businesses returned and find which restaurants are in SF    
    for business in yelp_data['businesses']:

        city = get_city(business["id"])
        
        #Since the results returned are 'best matches', filter out all the restaurants that have the same name and is located in SF
        if business['name'] in restaurant and city == "San Francisco":
            
            #Append a dictionary containing the fast food and it's rating
            restaurants_rating.append({"Fast Food": restaurant,
                               "Rating": business['rating'],
                                "Latitude": business['coordinates']['latitude'],
                                "Longitude": business['coordinates']['longitude']})

pprint(restaurants_rating)

In [ ]:
#Create a dataframe from the dictionary 
restaurant_pd = pd.DataFrame.from_dict(restaurants_rating)

#write to csv
restaurant_pd.to_csv("restaurant_results.csv", index=False)

In [4]:
#Read in csv
restaurant_pd = pd.read_csv("restaurant_results.csv")

#Group by the Fast Food
restaurant_group = restaurant_pd.groupby("Fast Food")

#Get the average rating for each restaurant
average_rating = restaurant_group["Rating"].mean()

#Get the number of restaurants
num_restaurants = restaurant_group["Fast Food"].count()

#Create a new dataframe to hold the average rating and number of restaurants
restaurant_df = pd.DataFrame({"Average Rating": average_rating.map("{:,.2f}".format),
                   "Number of Restaurants": num_restaurants})

#Sort by Average Rating descending
restaurant_df = restaurant_df.sort_values("Average Rating", ascending=0)

#Retrieve the columns in the order we want 
restaurant_df = restaurant_df[["Average Rating","Number of Restaurants"]]

#reset the index
restaurant_df = restaurant_df.reset_index()

#Print out the final dataframe
restaurant_df

,Fast Food,Average Rating,Number of Restaurants
0,In-N-Out Burger,4.00,1
1,McDonald's,2.32,14
2,Carl's Jr.,2.00,2
3,Burger King,1.95,11


In [31]:
import gmaps
from config import gmap_key
#Read in csv
restaurant_pd = pd.read_csv("restaurant_results.csv")
restaurant_pd.head()

mcdonalds = restaurant_pd.loc[restaurant_pd["Fast Food"] == "McDonald's"]
innout = restaurant_pd.loc[restaurant_pd["Fast Food"] == "In-N-Out Burger"]
carlsjr = restaurant_pd.loc[restaurant_pd["Fast Food"] == "Carl's Jr."]
burgerking = restaurant_pd.loc[restaurant_pd["Fast Food"] == "Burger King"]


mcdonalds_locations = mcdonalds[["Latitude", "Longitude"]]
innout_locations = innout[["Latitude", "Longitude"]]
carlsjr_locations = carlsjr[["Latitude", "Longitude"]]
burgerking_locations = burgerking[["Latitude", "Longitude"]]


In [50]:
gmaps.configure(api_key=gmap_key)

#Set a base map of SF
san_francisco_coordinates = (37.773972, -122.431297)
fig = gmaps.figure(center=san_francisco_coordinates, zoom_level=11.5)

#Add mcdonalds locations
mcdonalds_layer = gmaps.symbol_layer(
            mcdonalds_locations, fill_color="green", stroke_color="green", scale=4, fill_opacity=0.3)
fig.add_layer(mcdonalds_layer)

#Add In N Out Locations
innout_layer = gmaps.symbol_layer(
            innout_locations, fill_color="red", stroke_color="red", scale=4, fill_opacity=0.3)
fig.add_layer(innout_layer)

#Add CarlsJr Locations
carlsjr_layer = gmaps.symbol_layer(
            carlsjr_locations, fill_color="black", stroke_color="black", scale=4, fill_opacity=0.3)
fig.add_layer(carlsjr_layer)

#Add BurgerKing Locations
burgerking_layer = gmaps.symbol_layer(
            burgerking_locations,fill_color="blue", stroke_color="blue", scale=4, fill_opacity=0.2)
fig.add_layer(burgerking_layer)
#fig.add_layer(gmaps.heatmap_layer(mcdonalds_locations))
fig



Figure(layout=FigureLayout(height='420px'))